In [ ]:
#%pip install nltk

In [ ]:
import json
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np


# 📌 Cargar dataset original
file_path = "Ecommerce_FAQ_Chatbot_dataset.json"
with open(file_path, "r", encoding="utf-8") as file:
    dataset = json.load(file)["questions"]

# 📌 Cargar modelo de embeddings para evaluar similitud semántica
#embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",model_kwargs={"device": "cuda"})

# 📌 API del chatbot (debe estar corriendo)
API_URL = "http://127.0.0.1:8000/ask"

# 📌 Evaluar chatbot en preguntas del dataset
total_bleu, total_cosine = 0, 0
num_samples = 10  # Número de preguntas para la evaluación

for sample in dataset[:num_samples]:  # Usamos solo algunas preguntas para prueba
    question = sample["question"]
    expected_answer = sample["answer"]

    # 📌 Enviar pregunta al chatbot
    response = requests.post(API_URL, json={"question": question})
    generated_answer = response.text if response.status_code == 200 else ""

    # 📌 Calcular BLEU Score
    bleu_score = sentence_bleu([expected_answer.split()], generated_answer.split())

    # 📌 Calcular Cosine Similarity con `embed_query()`
    expected_embedding = embedding_model.embed_query(expected_answer)
    generated_embedding = embedding_model.embed_query(generated_answer)

    # Convertir a formato numpy (LangChain devuelve listas)
    expected_embedding = np.array(expected_embedding).reshape(1, -1)
    generated_embedding = np.array(generated_embedding).reshape(1, -1)

    cosine_score = cosine_similarity(expected_embedding, generated_embedding)[0][0]

    total_bleu += bleu_score
    total_cosine += cosine_score

    # 📌 Mostrar resultados para cada pregunta
    print(f"🔹 Question: {question}")
    print(f"✅ Expected Answer: {expected_answer}")
    print(f"🤖 Generated Answer: {generated_answer}")
    print(f"📊 BLEU Score: {bleu_score:.4f} | Cosine Similarity: {cosine_score:.4f}")
    print("-" * 80)

# 📌 Calcular métricas promedio
avg_bleu = total_bleu / num_samples
avg_cosine = total_cosine / num_samples
print(f"📌 Average BLEU Score: {avg_bleu:.4f}")
print(f"📌 Average Cosine Similarity: {avg_cosine:.4f}")


🔹 Question: How can I create an account?
✅ Expected Answer: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.
🤖 Generated Answer: "To create an account on our website, follow these steps:  1. Click on \"Mi cuenta\" (My Account) at the top right corner of the page. 2. Select \"Crear cuenta\" (Create Account). 3. Fill in your information, including name, email address, and password. 4. Review and agree to our terms and conditions. 5. Submit your application.  Please note that you will receive an email to verify your account after creating it."
📊 BLEU Score: 0.0661 | Cosine Similarity: 0.7194
--------------------------------------------------------------------------------


c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


🔹 Question: What payment methods do you accept?
✅ Expected Answer: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
🤖 Generated Answer: "We accept various payment methods to ensure your shopping experience is convenient and seamless. Our accepted payment methods include:  - Credit cards (Visa, Mastercard, American Express) - Debit cards - PayPal - Bank transfer (for select countries)  Please note that availability of certain payment options may vary depending on your location. If you have any specific questions or concerns about payment, feel free to ask!"
📊 BLEU Score: 0.0000 | Cosine Similarity: 0.7768
--------------------------------------------------------------------------------


c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


🔹 Question: How can I track my order?
✅ Expected Answer: You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.
🤖 Generated Answer: "To track your order, please follow these steps:  1. Log in to your account on our website. 2. Click on \"Orders\" or \"My Account\" (this may vary depending on our website's design). 3. Locate the order you wish to track and click on it. 4. Look for a tracking number or a link that says \"Track Order.\" 5. Enter the tracking number in the provided field, usually starting with \"RKP-\" or \"TNT-\".  If you're unable to find your tracking information, please contact our customer service team and we'll be happy to assist you.  (If the question is in Spanish, my answer will be in Spanish as well.)"
📊 BLEU Score: 0.0000 | Cosine Similarity: 0.7978
--------------------------------------------------------------------------------
🔹 Question: What is your

c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using 

🔹 Question: How long does shipping take?
✅ Expected Answer: Shipping times vary depending on the destination and the shipping method chosen. Standard shipping usually takes 3-5 business days, while express shipping can take 1-2 business days.
🤖 Generated Answer: "We ship orders within 24-48 hours of receipt and typically arrive within 3-7 business days for domestic orders and 10-15 business days for international orders. Please note that delivery times may vary due to circumstances beyond our control, such as weather or customs delays. If you have a specific question about your order's status, please feel free to contact us directly."
📊 BLEU Score: 0.0000 | Cosine Similarity: 0.8170
--------------------------------------------------------------------------------


c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\wesgb\.conda\envs\chatbot_nlp_rag\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


🔹 Question: Do you offer international shipping?
✅ Expected Answer: Yes, we offer international shipping to select countries. The availability and shipping costs will be calculated during the checkout process based on your location.
🤖 Generated Answer: "Yes, we do offer international shipping. Our website is accessible worldwide and we ship to most countries. However, please note that some items may not be available for international shipping due to size or weight restrictions, and delivery times may vary depending on the destination country.  If you're interested in purchasing an item that's eligible for international shipping, simply proceed with your order as normal and we'll take care of the rest. We'll provide you with tracking information so you can stay up-to-date on the status of your shipment."
📊 BLEU Score: 0.0000 | Cosine Similarity: 0.8374
--------------------------------------------------------------------------------
🔹 Question: What should I do if my package is lost or d